In [21]:
from selenium import webdriver
from time import sleep 
import requests
from pyquery import PyQuery as pq
import random
import datetime
import pytz
from datetime import datetime, time

randomize = True
hiValue = 0
lowValue = 0


def is_time_between(begin_time, end_time, time_to_test):
    time_to_test = time_to_test.time()
    if begin_time < end_time:
        return time_to_test >= begin_time and time_to_test <= end_time
    else: # crosses midnight
        return time_to_test >= begin_time or time_to_test <= end_time

def isUSMarketOpen():
    if randomize!= True:
        from datetime import datetime, time
        # def checkUSMarketOpen():
        utc = pytz.utc
        utc_dt = datetime(2002, 10, 27, 6, 0, 0, tzinfo=utc)
        eastern = pytz.timezone('US/Eastern')
        loc_dt = utc_dt.astimezone(eastern)
        fmt = '%Y-%m-%d %H:%M:%S %Z%z'
        ct = loc_dt.strftime(fmt)
        if(loc_dt.weekday()!=5 and loc_dt.weekday()!=6 and is_time_between(time(9,30), time(16,0),loc_dt)):
            return True
        else:
            return False
    else:
        return True

def mail():
    if randomize!= True:
        import smtplib, ssl
        port = 587  # For starttls
        smtp_server = "smtp.gmail.com"
        sender_email = "my@gmail.com"
        receiver_email = "my@gmail.com"
        password = ""
        message = """\
        Subject: Hi there

        This message is sent from Python."""
        # Create a secure SSL context
        context = ssl.create_default_context()

        # Try to log in to server and send email
        try:
            server = smtplib.SMTP(smtp_server,port)
            server.ehlo() # Can be omitted
            server.starttls(context=context) # Secure the connection
            server.ehlo() # Can be omitted
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message)

        except Exception as e:
            # Print any error messages to stdout
            print(e)
        finally:
            server.quit() 
        
def sendBuyAlert(priceChange):
    p = priceChange*100
    print('price has went down {}% Buy Now'.format(p))
    mail()

def sendSellAlert(priceChange):
    p = priceChange*100
    print('price has went up {}% Sell Now'.format(p))
    mail()



try:
   
    ticker = input('Enter a stock symbol ')
    percen = input('Enter a percentage to send alert to buy or sell ie: 2-> 2%')
    percentage = (float(percen))/100 
    if ticker != '':
        while True:
            response = requests.get("https://money.cnn.com/quote/quote.html?symb=" + ticker)
            doc = pq(response.text)
            c = doc('td.wsod_last >span').text()
            currentPrice = float(c)
            
            if randomize:
                currentPrice = random.randint(1,10)
    
            print('currentPrice:' + str(currentPrice))
            print('lowPrice:' + str(lowValue))
            print('hiPrice:' + str(hiValue))

            # first check
            if(hiValue==0 and lowValue==0):
                    hiValue = currentPrice
                    lowValue = currentPrice
            
            elif currentPrice < lowValue:
                priceChange = (hiValue-currentPrice)/currentPrice
                if ((hiValue-currentPrice)/currentPrice) > percentage:
                    sendBuyAlert(priceChange)
                    hiValue = currentPrice #if the buy alert is sent, then reset the hivalue to track for another drop
                    break
                lowValue = currentPrice #if price didnt cross the threshold, just set the low to currentvalue 
                
            
            elif currentPrice > hiValue:
                priceChange = ((currentPrice-lowValue)/lowValue) 
                if ((currentPrice-lowValue)/lowValue) > percentage:
                    sendSellAlert(priceChange)
                    lowValue = currentPrice #if the sell alert is sent, then reset the lowValue to track for another surge
                    break
                hiValue = currentPrice #if price didnt cross the threshold, just set the Hi to currentvalue 
            if(randomize):
                sleep(1) #sec
            else:
                sleep(2)
except EOFError:
    ticker = ''

Enter a stock symbol gs
Enter a percentage to send alert to buy or sell ie: 2-> 2%500
currentPrice:8
lowPrice:0
hiPrice:0
currentPrice:4
lowPrice:8
hiPrice:8
currentPrice:5
lowPrice:4
hiPrice:8
currentPrice:4
lowPrice:4
hiPrice:8
currentPrice:3
lowPrice:4
hiPrice:8
currentPrice:8
lowPrice:3
hiPrice:8
currentPrice:1
lowPrice:3
hiPrice:8
price has went down 700.0% Buy Now
